# **4. ML - Classification**

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

# Cargar el archivo CSV
df = pd.read_csv('../data/processed/campana_marketing.csv', parse_dates=['Dt_Customer'])

# Convertir la columna 'Income' de float64 a int64
df['Income'] = df['Income'].astype('int64')

## Nuestra variable objetivo será `AcceptedCmp`: 0 no se ha aceptado ninguna campaña y 1 se ha aceptado al menos 1 de las 5 campañas

In [14]:
# 1. Correlación con el Target
correlation_matrix = df.corr(numeric_only=True)
correlation_with_target = correlation_matrix['AcceptedCmp'].sort_values(ascending=False)
correlation_with_target

AcceptedCmp            1.000000
AcceptedCmp4           0.553782
AcceptedCmp3           0.551957
AcceptedCmp5           0.548292
AcceptedCmp1           0.510624
MntWines               0.465034
Spent                  0.412466
Response               0.367401
Income                 0.315024
NumCatalogPurchases    0.313291
MntMeatProducts        0.274626
AcceptedCmp2           0.229464
NumWebPurchases        0.213228
MntGoldProds           0.190782
NumStorePurchases      0.186765
MntFishProducts        0.160046
MntSweetProducts       0.159584
MntFruits              0.126553
Education              0.047823
Age                    0.030778
Marital_Status        -0.000198
Days                  -0.013751
Seniority             -0.013751
Recency               -0.017745
Complain              -0.027016
Year_Birth            -0.030778
ID                    -0.041206
NumDealsPurchases     -0.086570
Teenhome              -0.099415
NumWebVisitsMonth     -0.125986
Kidhome               -0.203024
Child_Ho

## Selección de variables X e Y

1. Determinamos que las variables con una correlación entre 0.2 y -0.2 quedarán excluidas de nuestro entrenamiento, ya que se encuentran entorno al 0 y parecen que no afectan a la variable objetivo y así aligeramos el proceso de entrenamiento.
2. Usando una **list comprehension** eliminamos las características `AcceptedCmp1, AcceptedCmp2, AcceptedCmp3, AcceptedCmp4 y AcceptedCmp5`, ya que son subproducto de AcceptedCmp.

In [15]:
# Filtrar correlaciones fuera del rango [-0.2, 0.2]
filtered_correlations = correlation_with_target[
    correlation_with_target.abs() > 0.2
].sort_values(ascending=False)

# Lista final de features, excluyendo las variables derivadas de 'AcceptedCmp'
excluded_features = ['AcceptedCmp', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
features = [f for f in filtered_correlations.index if f not in excluded_features]

print("Características seleccionadas:", features)

Características seleccionadas: ['MntWines', 'Spent', 'Response', 'Income', 'NumCatalogPurchases', 'MntMeatProducts', 'NumWebPurchases', 'Kidhome', 'Child_Home']


Definimos las variables

In [16]:
X = df[features]
y = df['AcceptedCmp']

## División de datos
Cogeremos el 20% para test y el 80% para training

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Tratamiento del Desbalance de Clases
Como vimos en [**EDA**](04_explore_data.ipynb), al final del análisis no gráfico, `AcceptedCmp` tiene la clase está desbalanceada.
En este escenario, la clase minoritaria (en nuestro caso, la clase 1) está significativamente menos representada que la clase mayoritaria (la clase 0). Esto puede generar varios problemas al entrenar un modelo de clasificación.

### Rebalancear clases
Optamos por hacer un Submuestreo (**Undersampling**), reduciendo el número de ejemplos de la clase mayoritaria.

Dada la significativa disparidad en la representación de las clases en nuestro conjunto de datos, con una marcada predominancia de la clase mayoritaria (clase 0), se ha optado por aplicar la técnica de submuestreo (undersampling). El objetivo principal de esta estrategia es reducir el número de instancias de la clase mayoritaria para mitigar el sesgo del modelo hacia esta clase y mejorar su capacidad para aprender y predecir la clase minoritaria (clase 1) de manera más efectiva.

In [ ]:
# Aplicar undersampling para balancear las clases
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

## Selección de modelos
Para el Desbalance de Clases, ajustamos pesos en el modelo (más fácil y eficiente) con `class_weight='balanced`. 

Como los modelos KNN y Neural Network no son compatibles para que se ajusten, aquí sí que aplicaremos el undersampling que se ha hecho previamente.

In [20]:
# Modelos con ajuste de pesos para modelos compatibles
models = {
    'Logistic Regression': (LogisticRegression(max_iter=500, class_weight='balanced'), X_train, y_train),
    'KNN': (KNeighborsClassifier(), X_resampled, y_resampled),
    'Random Forest': (RandomForestClassifier(class_weight='balanced'), X_train, y_train),
    'SVM': (SVC(probability=True, class_weight='balanced'), X_train, y_train),
    'Neural Network': (MLPClassifier(), X_resampled, y_resampled)
}

In [ ]:
# Hiperparámetros para GridSearchCV
param_grid = {
    'Logistic Regression': {'C': [0.01, 0.1, 1, 10, 100]},
    'KNN': {'n_neighbors': [3, 5, 7, 9, 11]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_features': ['sqrt', 'log2'], 'max_depth': [3, 5, 10]},
    'SVM': {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]},
    'Neural Network': {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.001, 0.01, 0.05],
        'learning_rate': ['constant','adaptive']
    }
}

results = {}
best_models = {}

for model_name, (model, X_data, y_data) in models.items():
    print(f"\nEntrenando {model_name}...")

    grid_search = GridSearchCV(model, param_grid[model_name], scoring='roc_auc', n_jobs=-1, cv=5)
    grid_search.fit(X_data, y_data)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    results[model_name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba),
        'Best Params': best_params
    }
    best_models[model_name] = best_model

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results).T
results_df[['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']] = results_df[['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']].astype(float)

print("\nResultados de los modelos:")
print(results_df)

# Determinar el mejor modelo basado en diferentes métricas
best_metrics = {metric: results_df[metric].idxmax() for metric in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']}

print("\n🔹 Mejores modelos por métrica:")
for metric, model in best_metrics.items():
    print(f"Mejor modelo basado en {metric}: {model} - {results_df.at[model, 'Best Params']}")